In [1]:
# !pip -q install diffusers
# !pip -q install transformers
# !pip -q install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.9/934.9 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 19.4 MB/s eta 0:00:00


In [2]:
import torch
import PIL
import diffusers
from diffusers import AutoencoderKL, DDPMScheduler, DiffusionPipeline, UNet2DConditionModel
from diffusers.loaders import AttnProcsLayers
from diffusers.models.attention_processor import LoRAAttnProcessor
from diffusers.optimization import get_scheduler
from diffusers.utils import check_min_version, is_wandb_available
from diffusers.utils.import_utils import is_xformers_available

In [3]:
pretrained_model_name_or_path = "runwayml/stable-diffusion-v1-5"
weight_dtype = torch.float16
device = torch.device("cuda:0")
# device = torch.device("mps")
output_dir = 'lora_output'

In [4]:
pipeline = DiffusionPipeline.from_pretrained(
    pretrained_model_name_or_path, safety_checker = None, revision=None, torch_dtype=weight_dtype
)

pipeline = pipeline.to(device)
# pipeline.enable_attention_slicing()

pipeline.unet.load_attn_procs(output_dir)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_errors.py:259 in                  │
│ hf_raise_for_status                                                                              │
│                                                                                                  │
│   256 │   </Tip>                                                                                 │
│   257 │   """                                                                                    │
│   258 │   try:                                                                                   │
│ ❱ 259 │   │   response.raise_for_status()                                                        │
│   260 │   except HTTPError as e:                                                                 │
│   261 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   262                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/requests/models.py:960 in raise_for_status               │
│                                                                                                  │
│   957 │   │   │   http_error_msg = u'%s Server Error: %s for url: %s' % (self.status_code, rea   │
│   958 │   │                                                                                      │
│   959 │   │   if http_error_msg:                                                                 │
│ ❱ 960 │   │   │   raise HTTPError(http_error_msg, response=self)                                 │
│   961 │                                                                                          │
│   962 │   def close(self):                                                                       │
│   963 │   │   """Releases the connection back to the pool. Once this method has been             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 401 Client Error: Unauthorized for url: 
https://huggingface.co/lora_output/resolve/main/pytorch_lora_weights.bin

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/diffusers/utils/hub_utils.py:311 in _get_model_file      │
│                                                                                                  │
│   308 │   │   │   │   )                                                                          │
│   309 │   │   try:                                                                               │
│   310 │   │   │   # 2. Load model file as usual                                                  │
│ ❱ 311 │   │   │   model_file = hf_hub_download(                                                  │
│   312 │   │   │   │   pretrained_model_name_or_path,                                             │
│   313 │   │   │   │   filename=weights_name,                                                     │
│   314 │   │   │   │   cache_dir=cache_dir,                                                       │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:120 in _inner_fn    │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   119 │   │                                                      

# Some testings

In [ ]:
noise_scheduler = DDPMScheduler.from_pretrained(pretrained_model_name_or_path, subfolder="scheduler")
noise_scheduler

DDPMScheduler {
  "_class_name": "DDPMScheduler",
  "_diffusers_version": "0.17.0.dev0",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "clip_sample_range": 1.0,
  "dynamic_thresholding_ratio": 0.995,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "sample_max_value": 1.0,
  "set_alpha_to_one": false,
  "skip_prk_steps": true,
  "steps_offset": 1,
  "thresholding": false,
  "trained_betas": null,
  "variance_type": "fixed_small"
}

In [ ]:
seed = 25238
generator = torch.Generator(device = device).manual_seed(seed)
validation_prompt = 'a scribble of cat'
lora_scale = 0.5
image = pipeline(validation_prompt, num_inference_steps=30, guidance_scale=7.5, cross_attention_kwargs={"scale": lora_scale}).images[0]
image

: 

: 

# Output image grid

In [ ]:
# # try different lora_scale values
# # 0 - 1 with step 0.1
# from matplotlib import pyplot as plt
# # draw grid
# validation_prompt = 'a scribble of cat'
# # seed = 25238
# # generator = torch.Generator(device='cpu').manual_seed(seed)

# fig, axs = plt.subplots(11, 11, figsize=(20, 20))
# for lora_scale in range(0, 10, 1):
#     lora_scale = lora_scale / 10
#     image = pipeline(validation_prompt, num_inference_steps=30, guidance_scale=7.5, cross_attention_kwargs={"scale": lora_scale}).to(device).images[0]
#     # image.save(f"lora_output/lora_scale_{lora_scale}.png")
#     axs[lora_scale // 11, lora_scale % 11].imshow(image)



: 

: 